<a href="https://colab.research.google.com/github/sbbwoy/CodeStudy/blob/main/%EC%9D%B4%EC%83%81%EC%B9%98%ED%83%90%EC%A7%80_DACON_%EB%B3%B4%EC%95%88_%EC%9C%84%ED%97%98%EB%8F%84_%EC%98%88%EC%B8%A1_AI_1%EC%9C%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://dacon.io/competitions/official/235717/codeshare/2679?page=1&dtype=vote 

# 접근 방식

Baseline의 Randomforest 지도학습 결과를 보았을 때 0.997 정도의 매우 높은 성능을 보여 Level7만 잘 구별한다면 좋은 성능을 얻을 수 있을 것이라고 생각했습니다.

7을 구별하기 위해서 비지도 학습으로 접근을 시도했고 오토인코더, one-class-svm, isolated-randomforest 등의 비지도 학습 방법을 적용해 보았으나 기존과 단어 몇개만 다른 경우들이 7 특성 떄문인지 성능이 좋지 않았습니다. 그 중에서 오토인코더가 성능이 괜찮았는데 "렛서펜더"님이 오토인코더 관련 코드를 올려주셨습니다. 
https://dacon.io/competitions/official/235717/codeshare/2677?page=1&dtype=recent


결론적으로 로그 데이터의 전처리를 진행하고 Baseline에서 Validation을 통해 등급별로 threshold를 상세 설정한 점수가 가장 높게 나왔습니다.Threshold 를 잡아서 7을 잡았지만, Train 문장과 완전히 동일한 문장이 존재하는지 여부를 체크해주는 컬럼을 통해 문장 전체가 완전히 동일한 경우에는 Threshold 에 걸리지 않도록 했습니다. 

마지막으로 점수를 조금 향상하기 위해서 7 등급 이외의 등급은 전처리가 덜 진행된 데이터를 활용하여 베이스라인 지도학습을 다시 진행했습니다. (지나친 전처리는 0~6등급 지도 학습 분류에 악영향이라고 판단) 


실제 서비스에서 Line by Line으로 들어왔을 때의 분류방법:
1. 로그 문장 데이터 전처리 진행 (Cleaning)
2. train 데이터와 완전히 동일한 문장이 있는지 확인하여 서로 다른 모델 적용(A방법, B방법)
3. 동일한 문장이 있다면 0~6 등급 지도 학습된 모델로 Predict(Threshold 적용 X) (A방법)
4. 동일한 문장이 없다면 0~6 등급 지도 학습된 모델로 Predict 이후에 설정된 Threshold 를 통해 7은 따로 분류 (B방법)
5. 4번에서 Threshold를 통해 걸러지지 않는다면 3번과 동일한 모델로 predict 적용 (A방법)


[참고]  
https://dacon.io/competitions/official/235717/codeshare/2536?page=1&dtype=recent (대회 베이스라인)  
https://dacon.io/competitions/official/235717/codeshare/2539?page=1&dtype=recent (10duck 님의 데이터 전처리 및 EDA)  
https://dacon.io/competitions/official/235717/support/403102?page=1&dtype=recent (taegu 님의 threshold 조정 문의 글)

# Data & Libaray IMPORT

In [2]:
!pip install dask[dataframe] --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 141 kB 4.6 MB/s 


In [3]:
import pandas as pd
import numpy as np
import os 
import pandas as pds
from dask import dataframe
import re
import numpy as np
import seaborn as sbn
from matplotlib import pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import wordnet

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
path = '/content/drive/MyDrive/22-여름 스터디/로그 분석을 통한 보안 위험도 예측 AI 경진대회_data/'

In [6]:
train = pd.read_csv(path+"train.csv")
test= pd.read_csv(path+"test.csv")

In [7]:
train.head()

,id,level,full_log
0,0,0,"Sep 24 10:02:22 localhost kibana: {""type"":""err..."
1,1,0,Feb 8 16:21:00 localhost logstash: [2021-02-0...
2,2,0,"Jan 13 01:50:40 localhost kibana: {""type"":""err..."
3,3,0,"Jan 4 10:18:31 localhost kibana: {""type"":""err..."
4,4,1,type=SYSCALL msg=audit(1603094402.016:52981): ...


In [8]:
test.head()

,id,full_log
0,1000000,"Feb 8 15:47:26 localhost kibana: {""type"":""err..."
1,1000001,"Sep 24 03:46:39 localhost kibana: {""type"":""err..."
2,1000002,type=SYSCALL msg=audit(1611888200.428:210563):...
3,1000003,"Jan 18 11:24:06 localhost kibana: {""type"":""err..."
4,1000004,type=SYSCALL msg=audit(1603081202.050:46851): ...


In [9]:
train.iloc[:,-1][0]

'Sep 24 10:02:22 localhost kibana: {"type":"error","@timestamp":"2020-09-24T01:02:22Z","tags":["warning","stats-collection"],"pid":6458,"level":"error","error":{"message":"No Living connections","name":"Error","stack":"Error: No Living connections\\n    at sendReqWithConnection (/usr/share/kibana/node_modules/elasticsearch/src/lib/transport.js:226:15)\\n    at next (/usr/share/kibana/node_modules/elasticsearch/src/lib/connection_pool.js:214:7)\\n    at process._tickCallback (internal/process/next_tick.js:61:11)"},"message":"No Living connections"}'

- 딕셔너리 형태... 까다롭다

In [10]:
len(test)

1418916

# EDA
### 1. 다양한 기호 및 숫자 제거
의미 없는 숫자와 기호들이 많아서 분류를 방해한다고 판단

In [11]:
lit = 'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ'
def mask(tt):
    tt=tt.apply(lambda x: re.sub(r'(\\n)',' ',x))
    tt=tt.apply(lambda x: re.sub(r'[^a-zA-Zㄱ-ㅣ가-힣0-9:=\s\(\)./,\<\>]+',' ',x))
    #tt=tt.apply(lambda x: re.sub(r' ?(?P<note>[:=\(\)./,\<\>]) ?', ' \g<note> ', x))
    tt=tt.apply(lambda x: re.sub(r'[0-9]+',' ',x))
    tt=tt.apply(lambda x: re.sub(r"':/()",' ',x))
    tt=tt.apply(lambda x: re.sub(r':',' ',x))
    tt=tt.apply(lambda x: re.sub(r',',' ',x))
    # = tt.apply(lambda x: re.sub(r'(',' ',x))
    #t = tt.apply(lambda x: re.sub(r')',' ',x))
    tt=tt.apply(lambda x: re.sub(r'[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]',' ',x))
    for st in lit:
        st = " "+st + " "
        tt=tt.apply(lambda x: re.sub(st,' ',x))
    tt=tt.apply(lambda x: re.sub(r'\s+',' ',x))
    
    return tt

- 뭐하는 함수지...? 

In [12]:
train['pre_log'] = mask(train.full_log)
test['pre_log'] = mask(test.full_log)

In [13]:
train['pre_log']

0         Sep localhost kibana type error timestamp tags...
1         Feb localhost logstash INFO logstash outputs e...
2         Jan localhost kibana type error timestamp tags...
3         Jan localhost kibana type error timestamp tags...
4         type SYSCALL msg audit arch syscall success ye...
                                ...                        
472967                        Feb localhost logstash error 
472968    type SYSCALL msg audit arch syscall success ye...
472969    Oct localhost kibana type log timestamp tags e...
472970    Jan localhost kibana type error timestamp tags...
472971    Oct localhost kibana type error timestamp tags...
Name: pre_log, Length: 472972, dtype: object

- 특수문자 -> 띄어쓰기

### 2. 영어 단어가 아니거나 3글자 미만인지인 경우에는 삭제
Validation 7등급을 보았을 때, 단어들이 의미가 있는 영어 단어들이 새로 들어오는 경향이 있다고 판단했습니다.

In [14]:
#포함되어 있는 단어 모음 생성
def count_word(data):
    tem = list(data['pre_log'].str.split(" "))  #공백으로 분리해서
    all_word = []                               
    for word in tem:
        all_word.extend(word)                   #all_word에 넣네
    words = pd.Series(all_word)
    return words.value_counts()                 #단어 개수 반환

In [15]:
train_count = count_word(train)
test_count = count_word(test)

In [16]:
train_words = list(set(train_count.index))
test_words = list(set(test_count.index))

- 중복 단어 없앤 듯

In [17]:
#영어 단어 + 3글자 이상인 경우만 True
from nltk.corpus import words
def check_words(word_list: list):
    re = [False] * len(word_list)   
    for i,word in enumerate(word_list):
        if len(word) < 3:
            continue
        word = word.lower()
        if word in words.words():
            re[i] = True
    return re

word_list = ['Promiscuous', 'ab', 'nihongo', 'abstract', 'pedo','gid']

- False가 1인 거 기억

In [19]:
import nltk
nltk.download('words')

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


True

In [20]:
#check_words는 단어인지를 확인해주고 결과를 t/f로 반환
check_words(word_list)

[True, False, False, True, False, True]

- 대박 신기함...

In [21]:
train_tf = check_words(train_words)
test_tf = check_words(test_words)

In [22]:
#단어인지 확인해주는 딕셔너리 만들어주기 (train_isword,test_isword)
train_isword = dict()
test_isword = dict()

total_words = list(train_words)
test_words = list(test_words)

for i in range(len(train_words)):
    train_isword[train_words[i].lower()] = train_tf[i]      
    
for i in range(len(test_words)):
    test_isword[test_words[i].lower()] = test_tf[i]
    

- train_isword, test_isword에 각각 단어인지 아닌지 t/f를 할당

### 3. Train, Test 에 각각 적용
##### 아래의 nltk 패키지를 이용해 영어가 아닌 단어를 제거하는 방법은 두가지를 소개 하였습니다. 결과는 동일하며 시간 측면에서 아래의 방식이 빠르기 때문에 느린 방법을 주석처리 하였습니다.  (위는 주석 처리)

- 모든 문장에 개별적으로 접근하여 영어가 아닌 단어를 제거하는 코드 (60~70 시간소요)
- 미리 사용된 단어들을 모아서 딕셔너리를 만들어서 사용한 경우(약 10분)

두가지 방법을 모두 소개한 이유는 아래의 방식은 TEST DATA를 모으는 행위인 Data Leakage 문제가 발생할 수 있다는 점 때문입니다. 

#단어를 하나씩 확인하는 방법(60~70 시간 이상 소요)
from nltk.corpus import words
def check_words(word_list: list):
    re = [False] * len(word_list)
    for i,word in enumerate(word_list):
        word = word.lower()
        if len(word)>2:
            if word in words.words():
                re[i] = True
    return re

def cutt(data):
    data = data.lower() 
    splited = data.split(" ")
    check = check_words(splited)
    c = np.array(splited)
    real_words = list(c[check])
    tem = " ".join(real_words)
    return tem

#valid['cut'] = valid['pre_log'].map(cutt)
train['cut']  =train['pre_log'].map(cutt)
test['cut']  =test['pre_log'].map(cutt)

In [23]:
import copy

def checking_train(data):
    re = [False] * len(data)
    for i,word in enumerate(data):
        if train_isword[word]:
            re[i] = True
    return re


def checking_test(data):
    re = [False] * len(data)
    for i,word in enumerate(data):
        if test_isword[word]:
            re[i] = True
    return re



def cutt_train(data):
    data = data.lower()
    splited = data.split(" ")
    check = checking_train(splited)
    c = np.array(splited)
    real_words = list(c[check])    
    tem = " ".join(real_words)
    #tem = tem.lower() 
    return tem

def cutt_test(data):
    data = data.lower()
    splited = data.split(" ")
    check = checking_test(splited)
    c = np.array(splited)
    real_words = list(c[check])  
    tem = " ".join(real_words)
    #tem = tem.lower()
    return tem

In [27]:
train['cut'] = train['pre_log'].map(cutt_train)
test['cut'] = test['pre_log'].map(cutt_test)

In [28]:
del train['full_log']
del test['full_log']

del train['pre_log']
del test['pre_log']


train.to_csv("sub_train3.csv",index=False)
test.to_csv("sub_test3.csv",index=False)


train.to_csv("only_train.csv",index =False)
test.to_csv("only_test.csv",index =False)

### 4. 나중에 사용될 inornot 리스트

train 데이터에 100퍼센트 동일한 문장이 있는지 확인해주는 변수 입니다. 

In [29]:
train_bag = train['cut'].unique()
len(train_bag)

6544

In [30]:
#inornot을 통해서 train안에 같은 문장이 있으면 0 없으면 1
from tqdm import tqdm
inornot = np.zeros(len(test))
for i in tqdm(range(len(test))):
    tem = test.iloc[i]['cut']
    if tem not in train_bag:
        inornot[i] = 1

100%|██████████| 1418916/1418916 [10:43<00:00, 2206.36it/s]


# 모델

train = pd.read_csv("sub_train3.csv")
test= pd.read_csv("sub_test3.csv")

In [31]:
train.head(20)

,id,level,cut
0,0,0,type error warning collection level error erro...
1,1,0,action with response code type unavailable exc...
2,2,0,type error warning collection level error erro...
3,3,0,type error warning collection level error erro...
4,4,1,type audit arch success yes exit gid suid none...
5,5,1,type audit arch success yes exit gid suid none...
6,6,0,type error warning collection level error erro...
7,7,0,warn resurrect connection dead instance but go...
8,8,0,type error warning collection level error erro...
9,9,0,type log warning message error living share no...


In [32]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier

#남은 단어가 없는 경우에는 'missing' 으로 대체해주었습니다.
train['cut'] = train['cut'].replace('','missing',regex=True)
test['cut'] = test['cut'].replace('','missing',regex=True)



train = train.fillna("missing")
test = test.fillna("missing")

train

,id,level,cut
0,0,0,type error warning collection level error erro...
1,1,0,action with response code type unavailable exc...
2,2,0,type error warning collection level error erro...
3,3,0,type error warning collection level error erro...
4,4,1,type audit arch success yes exit gid suid none...
...,...,...,...
472967,472967,0,error
472968,472968,1,type audit arch success yes exit gid suid none...
472969,472969,0,type log error task manager message poll for w...
472970,472970,0,type error warning collection level error erro...


In [33]:
train_text=list(train['cut'])
train_level=np.array(train['level'])

test_text=list(test['cut'])

In [34]:
vectorizer=CountVectorizer(analyzer="word", max_features=20000)
train_features=vectorizer.fit_transform(train_text)
test_features=vectorizer.transform(test_text)

In [35]:
forest=RandomForestClassifier(n_estimators=100,random_state = 1 )
forest.fit(train_features,train_level)

RandomForestClassifier(random_state=1)

In [36]:
results=forest.predict(test_features)
results_proba=forest.predict_proba(test_features)

In [37]:
#Validation을 통한 threshold 조절
results[np.where((np.max(results_proba, axis=1)<0.5) & (results == 0))[0]]=7
results[np.where((np.max(results_proba, axis=1)<0.5) & (results == 1))[0]]=7
results[np.where((np.max(results_proba, axis=1)<0.58) & (results == 2))[0]]=7
results[np.where((np.max(results_proba, axis=1)<0.95) & (results == 3))[0]]=7
results[np.where((np.max(results_proba, axis=1)<0.58) & (results == 4))[0]]=7
results[np.where((np.max(results_proba, axis=1)<0.58) & (results == 5))[0]]=7
results[np.where((np.max(results_proba, axis=1)<0.58) & (results == 6))[0]]=7

In [38]:
#Validation을 통한 threshold 조절
results[np.where((np.max(results_proba, axis=1)<0.94001) & (np.max(results_proba, axis=1) > 0.93999) & (results == 1))[0]]=7 
results[np.where((np.max(results_proba, axis=1)<0.611657) & (np.max(results_proba, axis=1) > 0.6116568) & (results == 0))[0]]=7
results[np.where((np.max(results_proba, axis=1)<0.571657) & (np.max(results_proba, axis=1) > 0.5716568) & (results == 0))[0]]=7
results[np.where((np.max(results_proba, axis=1)<0.68001) & (np.max(results_proba, axis=1) > 0.67999) & (results == 5))[0]]=7 


# 결과 저장

In [40]:
submission=pd.read_csv('/content/drive/MyDrive/22-여름 스터디/로그 분석을 통한 보안 위험도 예측 AI 경진대회_data/sample_submission.csv')
submission['level']=results
submission['level'].value_counts()

0    1002369
1     396443
3      12813
5       6418
7        780
2         34
4         34
6         25
Name: level, dtype: int64

In [41]:
submission.to_csv("tem_answer.csv",index = False)

# 추가작업
TEST 데이터에 동일한 문장이 있는지를 생성했던 컬럼을 추가해주는 작업입니다.

In [42]:
submission['inornot'] = pd.Series(inornot)

In [43]:
cal = submission[(submission['level']==7) & (submission['inornot'] == 0)].index
not7_id = list(test.iloc[cal]['id'])
pd.Series(not7_id).to_csv("not7_id.csv",index= False)

# 추가 작업

위의 방식으로 많이 전처리를 하면 7을 걸러내기 위한 threshold 를 잡기는 유리하지만, 지도 학습 자체의 성능은 떨어지는 것을 확인 하였습니다. 따라서, 7를 제외한 나머지 등급에는 전처리가 덜 진행된 Data로 지도학습 분류를 하였습니다. TfidfVectorizer이 더 성능이 우수하여 사용했습니다. 

In [49]:
path = '/content/drive/MyDrive/22-여름 스터디/로그 분석을 통한 보안 위험도 예측 AI 경진대회_data/'

In [50]:
train=pd.read_csv(path+'train.csv')
test=pd.read_csv(path+'test.csv')

In [51]:
train['full_log']=train['full_log'].str.replace(r'[0-9]', '<num>')
test['full_log']=test['full_log'].str.replace(r'[0-9]', '<num>')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  


In [52]:
train = train.fillna("missing")
test = test.fillna("missing")

In [53]:
train_text=list(train['full_log'])
train_level=np.array(train['level'])

test_text=list(test['full_log'])
#ids=list(test['id'])

In [54]:
vectorizer=TfidfVectorizer(analyzer="word", max_features=20000)
train_features=vectorizer.fit_transform(train_text)
test_features=vectorizer.transform(test_text)

In [55]:
forest=RandomForestClassifier(n_estimators=100,random_state = 1 )
forest.fit(train_features,train_level)

RandomForestClassifier(random_state=1)

In [56]:
results=forest.predict(test_features)
results_proba=forest.predict_proba(test_features)

In [59]:
submission=pd.read_csv(path+'sample_submission.csv')
submission['level']=results
submission['level'].value_counts()

0    1002770
1     396532
3      12997
5       6524
2         34
4         34
6         25
Name: level, dtype: int64

In [60]:
not7_id = pd.read_csv('not7_id.csv')
not7_id = list(not7_id['0'])

In [61]:
#위에서 잡은 7 합치기 & train에 있는 경우는 사용하지 않기
only = pd.read_csv("tem_answer.csv")
le7 = list(only[only['level']==7]['id'])
le7_update = list((set(le7))-set(not7_id))
print(len(le7))
print(len(le7_update))
idx = submission[submission['id'].isin(le7_update)].index
submission.iloc[idx] = 7
submission['level'].value_counts()

780
581


0    1002512
1     396348
3      12915
5       6467
7        581
2         34
4         34
6         25
Name: level, dtype: int64

In [63]:
who = submission.level
submission=pd.read_csv(path+'sample_submission.csv')
submission['level'] = who
submission['level'].value_counts()

0    1002512
1     396348
3      12915
5       6467
7        581
2         34
4         34
6         25
Name: level, dtype: int64

In [64]:
submission.to_csv("581_final_answer.csv",index = False)